In [1]:
import argparse
import json
import os
import sys
import warnings
from importlib import import_module
from pathlib import Path
from shutil import copy
from typing import Dict, List, Union

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchcontrib.optim import SWA

from data_utils import (Dataset_ASVspoof2019_train,
                        Dataset_ASVspoof2019_devNeval, genSpoof_list_prevDbs,
                        Dataset_train_prevDbs, Dataset_PrevDbs)
from evaluation import calculate_tDCF_EER
from utils import create_optimizer, seed_worker, set_seed, str_to_bool

warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
config = {
    "database_path": "/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/CMU/",
#     "asv_score_path": "ASVspoof2019_LA_asv_scores/ASVspoof2019.LA.asv.eval.gi.trl.scores.txt",
    "model_path": "./models/weights/AASIST-L.pth",
    "batch_size": 24,
    "num_epochs": 100,
    "loss": "CCE",
    "track": "LA",
    "eval_all_best": "True",
    "eval_output": "eval_scores_cmu_cmu.txt",
    "cudnn_deterministic_toggle": "True",
    "cudnn_benchmark_toggle": "False",
    "model_config": {
        "architecture": "AASIST",
        "nb_samp": 64600,
        "first_conv": 128,
        "filts": [70, [1, 32], [32, 32], [32, 24], [24, 24]],
        "gat_dims": [24, 32],
        "pool_ratios": [0.4, 0.5, 0.7, 0.5],
        "temperatures": [2.0, 2.0, 100.0, 100.0]
    },
    "optim_config": {
        "optimizer": "adam", 
        "amsgrad": "False",
        "base_lr": 0.0001,
        "lr_min": 0.000005,
        "betas": [0.9, 0.999],
        "weight_decay": 0.0001,
        "scheduler": "cosine"
    }
}


In [4]:
model_config = config["model_config"]
optim_config = config["optim_config"]
optim_config["epochs"] = config["num_epochs"]
# track = config["track"]

In [5]:
if "eval_all_best" not in config:
    config["eval_all_best"] = "True"
if "freq_aug" not in config:
    config["freq_aug"] = "False"

In [6]:
set_seed(1234, config)

In [7]:
# import random 
 
# fin = open("/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/protocols/cmu_train_protocol.txt", 'rb') 
# f90out = open("./protocols/cmu_split_train.txt", 'wb') 
# f10out = open("./protocols/cmu_split_dev.txt", 'wb') 
# for line in fin: 
#     r = random.random() 
#     if r < 0.90: 
#         f90out.write(line) 
#     else: 
#         f10out.write(line) 
# fin.close() 
# f90out.close() 
# f10out.close() 

In [8]:
config["database_path"]

'/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/CMU/'

In [9]:
output_dir = Path('./exp_result')
# prefix_2019 = "cmu.{}".format(track)
database_path = Path(config["database_path"])
dev_trial_path = Path('./protocols/cmu_split_dev.txt')
# (database_path /
#                     "ASVspoof2019_{}_cm_protocols/{}.cm.dev.trl.txt".format(
#                         track, prefix_2019))
eval_trial_path = Path('/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/protocols/cmu_test_protocol.txt')
# (
#     database_path /
#     "ASVspoof2019_{}_cm_protocols/{}.cm.eval.trl.txt".format(
#         track, prefix_2019))


In [10]:
# define model related paths
model_tag = "{}_{}_ep{}_bs{}".format(
    'CMU',
    'AASIST-L',
    config["num_epochs"], config["batch_size"])
# if args.comment:
#     model_tag = model_tag + "_{}".format(args.comment)
model_tag = output_dir / model_tag
model_save_path = model_tag / "weights"
eval_score_path = model_tag / config["eval_output"]
writer = SummaryWriter(model_tag)
os.makedirs(model_save_path, exist_ok=True)
# copy(args.config, model_tag / "config.conf")

In [11]:
def get_model(model_config: Dict, device: torch.device):
    """Define DNN model architecture"""
    module = import_module("models.{}".format(model_config["architecture"]))
    _model = getattr(module, "Model")
    model = _model(model_config).to(device)
    nb_params = sum([param.view(-1).size()[0] for param in model.parameters()])
    print("no. model params:{}".format(nb_params))

    return model

In [12]:
# set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device: {}".format(device))
if device == "cpu":
    raise ValueError("GPU not detected!")

# define model architecture
model = get_model(model_config, device)

# # define dataloaders
# trn_loader, dev_loader, eval_loader = get_loader(
#     database_path, 1234, config)

Device: cuda
no. model params:85306


In [13]:
database_path

PosixPath('/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/CMU')

In [14]:
"""Make PyTorch DataLoaders for train / developement / evaluation"""
# track = config["track"]
# prefix_2019 = "ASVspoof2019.{}".format(track)

trn_database_path = database_path / 'train'#"ASVspoof2019_{}_train/".format(track)
dev_database_path = database_path / 'train'#"ASVspoof2019_{}_dev/".format(track)
eval_database_path = database_path / 'test'#"ASVspoof2019_{}_eval/".format(track)

trn_list_path = Path('./protocols/cmu_split_train.txt')
# (database_path /
#                  "ASVspoof2019_{}_cm_protocols/{}.cm.train.trn.txt".format(
#                      track, prefix_2019))
dev_trial_path = './protocols/cmu_split_dev.txt'
# (database_path /
#                   "ASVspoof2019_{}_cm_protocols/{}.cm.dev.trl.txt".format(
#                       track, prefix_2019))
eval_trial_path = '/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/protocols/cmu_test_protocol.txt'
# (
#     database_path /
#     "ASVspoof2019_{}_cm_protocols/{}.cm.eval.trl.txt".format(
#         track, prefix_2019))

d_label_trn, file_train = genSpoof_list_prevDbs(dir_meta=trn_list_path,
                                        is_train=True,
                                        is_eval=False)
print("no. training files:", len(file_train))

train_set = Dataset_train_prevDbs(list_IDs=file_train,
                                       labels=d_label_trn,
                                       base_dir=trn_database_path)
gen = torch.Generator()
gen.manual_seed(1234)
trn_loader = DataLoader(train_set,
                        batch_size=config["batch_size"],
                        shuffle=True,
                        drop_last=True,
                        pin_memory=True,
                        worker_init_fn=seed_worker,
                        generator=gen)

_, file_dev = genSpoof_list_prevDbs(dir_meta=dev_trial_path,
                            is_train=False,
                            is_eval=False)
print("no. validation files:", len(file_dev))

dev_set = Dataset_PrevDbs(list_IDs=file_dev,
                                        base_dir=dev_database_path)
dev_loader = DataLoader(dev_set,
                        batch_size=config["batch_size"],
                        shuffle=False,
                        drop_last=False,
                        pin_memory=True)

file_eval = genSpoof_list_prevDbs(dir_meta=eval_trial_path,
                          is_train=False,
                          is_eval=True)
eval_set = Dataset_PrevDbs(list_IDs=file_eval,
                                         base_dir=eval_database_path)
eval_loader = DataLoader(eval_set,
                         batch_size=config["batch_size"],
                         shuffle=False,
                         drop_last=False,
                         pin_memory=True)

# return trn_loader, dev_loader, eval_loader

no. training files: 987
no. validation files: 123


In [15]:
# get optimizer and scheduler
optim_config["steps_per_epoch"] = len(trn_loader)
optimizer, scheduler = create_optimizer(model.parameters(), optim_config)
optimizer_swa = SWA(optimizer)

best_dev_eer = 1.
best_eval_eer = 100.
best_dev_tdcf = 0.05
best_eval_tdcf = 1.
n_swa_update = 0  # number of snapshots of model to use in SWA
f_log = open(model_tag / "metric_log.txt", "a")
f_log.write("=" * 5 + "\n")

6

In [16]:
model_tag

PosixPath('exp_result/CMU_AASIST-L_ep100_bs24')

In [17]:
# make directory for metric logging
metric_path = model_tag / "metrics"
os.makedirs(metric_path, exist_ok=True)

In [18]:
def produce_evaluation_file(
    data_loader: DataLoader,
    model,
    device: torch.device,
    save_path: str,
    trial_path: str) -> None:
    """Perform evaluation and save the score to a file"""
    model.eval()
    with open(trial_path, "r") as f_trl:
        trial_lines = f_trl.readlines()
    fname_list = []
    score_list = []
    for batch_x, utt_id in data_loader:
        batch_x = batch_x.to(device)
        with torch.no_grad():
            _, batch_out = model(batch_x)
            batch_score = (batch_out[:, 1]).data.cpu().numpy().ravel()
        # add outputs
        fname_list.extend(utt_id)
        score_list.extend(batch_score.tolist())

    assert len(trial_lines) == len(fname_list) == len(score_list)
    with open(save_path, "w") as fh:
        for fn, sco, trl in zip(fname_list, score_list, trial_lines):
            utt_id, key = trl.strip().split(' ')
            assert fn == utt_id
            fh.write("{} {} {} \n".format(utt_id, key, sco))
    print("Scores saved to {}".format(save_path))


def train_epoch(
    trn_loader: DataLoader,
    model,
    optim: Union[torch.optim.SGD, torch.optim.Adam],
    device: torch.device,
    scheduler: torch.optim.lr_scheduler,
    config: argparse.Namespace):
    """Train the model for one epoch"""
    running_loss = 0
    num_total = 0.0
    ii = 0
    model.train()

    # set objective (Loss) functions
    weight = torch.FloatTensor([0.1, 0.9]).to(device)
    criterion = nn.CrossEntropyLoss(weight=weight)
    for batch_x, batch_y in trn_loader:
        batch_size = batch_x.size(0)
        num_total += batch_size
        ii += 1
        batch_x = batch_x.to(device)
        batch_y = batch_y.view(-1).type(torch.int64).to(device)
        _, batch_out = model(batch_x, Freq_aug=str_to_bool(config["freq_aug"]))
        batch_loss = criterion(batch_out, batch_y)
        running_loss += batch_loss.item() * batch_size
        optim.zero_grad()
        batch_loss.backward()
        optim.step()

        if config["optim_config"]["scheduler"] in ["cosine", "keras_decay"]:
            scheduler.step()
        elif scheduler is None:
            pass
        else:
            raise ValueError("scheduler error, got:{}".format(scheduler))

    running_loss /= num_total
    return running_loss

In [19]:
import numpy as np
from evaluation import compute_eer
def getEER(score_path):
    cm_data = np.genfromtxt(score_path, dtype=str)
    cm_keys = cm_data[:, 1]
    cm_scores = cm_data[:, 2].astype(float)
    bona_cm = cm_scores[cm_keys == 'bonafide']
    spoof_cm = cm_scores[cm_keys == 'spoof']
    eer_cm = compute_eer(bona_cm, spoof_cm)[0]
    return eer_cm * 100

In [21]:
# Training
for epoch in range(config["num_epochs"]):
    print("Start training epoch{:03d}".format(epoch))
    running_loss = train_epoch(trn_loader, model, optimizer, device,
                               scheduler, config)
    produce_evaluation_file(dev_loader, model, device,
                            metric_path/"dev_score.txt", dev_trial_path)
    dev_eer = getEER(metric_path/"dev_score.txt")
#     , dev_tdcf = calculate_tDCF_EER(
#         cm_scores_file=metric_path/"dev_score.txt",
#         asv_score_file=database_path/config["asv_score_path"],
#         output_file=metric_path/"dev_t-DCF_EER_{}epo.txt".format(epoch),
#         printout=False)
    print("DONE.\nLoss:{:.5f}, dev_eer: {:.3f}".format(
        running_loss, dev_eer))
    writer.add_scalar("loss", running_loss, epoch)
    writer.add_scalar("dev_eer", dev_eer, epoch)
#     writer.add_scalar("dev_tdcf", dev_tdcf, epoch)

#     best_dev_tdcf = min(dev_tdcf, best_dev_tdcf)
    if best_dev_eer >= dev_eer:
        print("best model find at epoch", epoch)
        best_dev_eer = dev_eer
        torch.save(model.state_dict(),
                   model_save_path / "epoch_{}_{:03.3f}.pth".format(epoch, dev_eer))

        # do evaluation whenever best model is renewed
        if str_to_bool(config["eval_all_best"]):
            produce_evaluation_file(eval_loader, model, device,
                                    eval_score_path, eval_trial_path)
            dev_eer = getEER(metric_path/"dev_score.txt")
            eval_eer = getEER(eval_score_path)
#             , eval_tdcf = calculate_tDCF_EER(
#                 cm_scores_file=eval_score_path,
#                 asv_score_file=database_path / config["asv_score_path"],
#                 output_file=metric_path /
#                 "t-DCF_EER_{:03d}epo.txt".format(epoch))

            log_text = "epoch{:03d}, ".format(epoch)
            if eval_eer < best_eval_eer:
                log_text += "best eer, {:.4f}%".format(eval_eer)
                best_eval_eer = eval_eer
                torch.save(model.state_dict(),
                           model_save_path / "best.pth")
#             if eval_tdcf < best_eval_tdcf:
#                 log_text += "best tdcf, {:.4f}".format(eval_tdcf)
#                 best_eval_tdcf = eval_tdcf
#                 torch.save(model.state_dict(),
#                            model_save_path / "best.pth")
            if len(log_text) > 0:
                print(log_text)
                f_log.write(log_text + "\n")

        print("Saving epoch {} for swa".format(epoch))
        optimizer_swa.update_swa()
        n_swa_update += 1
    writer.add_scalar("best_dev_eer", best_dev_eer, epoch)
    writer.add_scalar("best_dev_tdcf", best_dev_tdcf, epoch)

Start training epoch000
Scores saved to exp_result/CMU_AASIST-L_ep100_bs24/metrics/dev_score.txt
DONE.
Loss:0.08438, dev_eer: 1.629
Start training epoch001
Scores saved to exp_result/CMU_AASIST-L_ep100_bs24/metrics/dev_score.txt
DONE.
Loss:0.08104, dev_eer: 0.000
best model find at epoch 1
Scores saved to exp_result/CMU_AASIST-L_ep100_bs24/eval_scores_cmu_cmu.txt
epoch001, 
Saving epoch 1 for swa
Start training epoch002
Scores saved to exp_result/CMU_AASIST-L_ep100_bs24/metrics/dev_score.txt
DONE.
Loss:0.06457, dev_eer: 0.000
best model find at epoch 2
Scores saved to exp_result/CMU_AASIST-L_ep100_bs24/eval_scores_cmu_cmu.txt
epoch002, 
Saving epoch 2 for swa
Start training epoch003
Scores saved to exp_result/CMU_AASIST-L_ep100_bs24/metrics/dev_score.txt
DONE.
Loss:0.06005, dev_eer: 0.000
best model find at epoch 3
Scores saved to exp_result/CMU_AASIST-L_ep100_bs24/eval_scores_cmu_cmu.txt
epoch003, 
Saving epoch 3 for swa
Start training epoch004
Scores saved to exp_result/CMU_AASIST-L_

Scores saved to exp_result/CMU_AASIST-L_ep100_bs24/eval_scores_cmu_cmu.txt
epoch031, 
Saving epoch 31 for swa
Start training epoch032
Scores saved to exp_result/CMU_AASIST-L_ep100_bs24/metrics/dev_score.txt
DONE.
Loss:0.01475, dev_eer: 0.000
best model find at epoch 32
Scores saved to exp_result/CMU_AASIST-L_ep100_bs24/eval_scores_cmu_cmu.txt
epoch032, 
Saving epoch 32 for swa
Start training epoch033
Scores saved to exp_result/CMU_AASIST-L_ep100_bs24/metrics/dev_score.txt
DONE.
Loss:0.01071, dev_eer: 0.000
best model find at epoch 33
Scores saved to exp_result/CMU_AASIST-L_ep100_bs24/eval_scores_cmu_cmu.txt
epoch033, 
Saving epoch 33 for swa
Start training epoch034
Scores saved to exp_result/CMU_AASIST-L_ep100_bs24/metrics/dev_score.txt
DONE.
Loss:0.00903, dev_eer: 0.000
best model find at epoch 34
Scores saved to exp_result/CMU_AASIST-L_ep100_bs24/eval_scores_cmu_cmu.txt
epoch034, 
Saving epoch 34 for swa
Start training epoch035
Scores saved to exp_result/CMU_AASIST-L_ep100_bs24/metri

KeyboardInterrupt: 

In [22]:
print("Start final evaluation")
epoch += 1
if n_swa_update > 0:
    optimizer_swa.swap_swa_sgd()
    optimizer_swa.bn_update(trn_loader, model, device=device)
produce_evaluation_file(eval_loader, model, device, eval_score_path,
                        eval_trial_path)
eval_eer = getEER(eval_score_path)
# , eval_tdcf = calculate_tDCF_EER(cm_scores_file=eval_score_path,
#                                          asv_score_file=database_path /
#                                          config["asv_score_path"],
#                                          output_file=model_tag / "t-DCF_EER.txt")
f_log = open(model_tag / "metric_log.txt", "a")
f_log.write("=" * 5 + "\n")
f_log.write("EER: {:.3f}".format(eval_eer))
f_log.close()

torch.save(model.state_dict(),
           model_save_path / "swa.pth")

if eval_eer <= best_eval_eer:
    best_eval_eer = eval_eer
# if eval_tdcf <= best_eval_tdcf:
#     best_eval_tdcf = eval_tdcf
    torch.save(model.state_dict(),
               model_save_path / "best.pth")
print("Exp FIN. EER: {:.3f}".format(
    best_eval_eer))

Start final evaluation
Scores saved to exp_result/CMU_AASIST-L_ep100_bs24/eval_scores_cmu_cmu.txt
Exp FIN. EER: 0.000


# LJ

In [24]:
# list of dataset partitions
SET_PARTITION = ["trn", "eval"]

# list of countermeasure(CM) protocols
SET_CM_PROTOCOL = {
    "trn": "/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/protocols/lj_train_protocol.txt",
    "eval": "/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/protocols/lj_test_protocol.txt",
}

# directories of each dataset partition
SET_DIR = {
    "trn": "/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/LJ/train/",
    "eval": "/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/LJ/test/",
}

In [25]:
# Dataloader
meta_lines = open(SET_CM_PROTOCOL["eval"], "r").readlines()
utt_list = []
for line in meta_lines:
    tmp = line.strip().split(" ")

    utt = tmp[0]
    utt_list.append(utt)

base_dir = SET_DIR['eval']
dataset = Dataset_PrevDbs(utt_list, Path(base_dir))
lj_loader = DataLoader(
        dataset, batch_size=24, shuffle=False, drop_last=False, pin_memory=True
    )

In [26]:
model.eval()

Model(
  (conv_time): CONV()
  (first_bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=True)
  (drop_way): Dropout(p=0.2, inplace=True)
  (selu): SELU(inplace=True)
  (encoder): Sequential(
    (0): Sequential(
      (0): Residual_block(
        (conv1): Conv2d(1, 32, kernel_size=(2, 3), stride=(1, 1), padding=(1, 1))
        (selu): SELU(inplace=True)
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(32, 32, kernel_size=(2, 3), stride=(1, 1), padding=(0, 1))
        (conv_downsample): Conv2d(1, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
        (mp): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
      )
    )
    (1): Sequential(
      (0): Residual_block(
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(32, 32, kernel_size=(2, 3),

In [28]:
eval_score_path = Path('exp_result/CMU_AASIST-L_ep100_bs24/eval_scores_cmu_lj')

In [30]:
trial_path = SET_CM_PROTOCOL['eval']
with open(trial_path, "r") as f_trl:
    trial_lines = f_trl.readlines()
fname_list = []
score_list = []
for batch_x, utt_id in lj_loader:
    batch_x = batch_x.to(device)
    with torch.no_grad():
        _, batch_out = model(batch_x)
        batch_score = (batch_out[:, 1]).data.cpu().numpy().ravel()
    # add outputs
    fname_list.extend(utt_id)
    score_list.extend(batch_score.tolist())   
assert len(trial_lines) == len(fname_list) == len(score_list)
with open(eval_score_path, "w") as fh:
    for fn, sco, trl in zip(fname_list, score_list, trial_lines):
        utt_id, key = trl.strip().split(' ')
        assert fn == utt_id
        fh.write("{} {} {}\n".format(utt_id, key, sco))

In [31]:
eer_lj = getEER(eval_score_path)
print('LJ EER: ', eer_lj)

LJ EER:  36.734693877551024


# Libri

In [32]:
# list of dataset partitions
SET_PARTITION = ["trn", "eval"]

# list of countermeasure(CM) protocols
SET_CM_PROTOCOL = {
    "trn": "/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/protocols/libri_train_protocol.txt",
    "eval": "/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/protocols/libri_test_protocol.txt",
}

# directories of each dataset partition
SET_DIR = {
    "trn": "/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/Libri/train/",
    "eval": "/DATA/nfsshare/rishith/datasets/smaller_dbs/restructured/Libri/test/",
}

In [33]:
# Dataloader
meta_lines = open(SET_CM_PROTOCOL["eval"], "r").readlines()
utt_list = []
for line in meta_lines:
    tmp = line.strip().split(" ")

    utt = tmp[0]
    utt_list.append(utt)

base_dir = SET_DIR['eval']
dataset = Dataset_PrevDbs(utt_list, Path(base_dir))
libri_loader = DataLoader(
        dataset, batch_size=24, shuffle=False, drop_last=False, pin_memory=True
    )

In [34]:
eval_score_path = Path('exp_result/CMU_AASIST-L_ep100_bs24/eval_scores_cmu_libri')

In [35]:
trial_path = SET_CM_PROTOCOL['eval']
with open(trial_path, "r") as f_trl:
    trial_lines = f_trl.readlines()
fname_list = []
score_list = []
for batch_x, utt_id in libri_loader:
    batch_x = batch_x.to(device)
    with torch.no_grad():
        _, batch_out = model(batch_x)
        batch_score = (batch_out[:, 1]).data.cpu().numpy().ravel()
    # add outputs
    fname_list.extend(utt_id)
    score_list.extend(batch_score.tolist())   
assert len(trial_lines) == len(fname_list) == len(score_list)
with open(eval_score_path, "w") as fh:
    for fn, sco, trl in zip(fname_list, score_list, trial_lines):
        utt_id, key = trl.strip().split(' ')
        assert fn == utt_id
        fh.write("{} {} {}\n".format(utt_id, key, sco))

In [36]:
eer_libri = getEER(eval_score_path)
print('Libri EER: ', eer_libri)

Libri EER:  28.57142857142857


# ASVspoof 2019

In [42]:
eval_trial_path = Path('/DATA/nfsshare/rishith/datasets/asvSpoof2019/DS_10283_3336/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt')
eval_database_path = Path('/DATA/nfsshare/rishith/datasets/asvSpoof2019/DS_10283_3336/LA/ASVspoof2019_LA_eval/')

In [51]:
def genSpoof_list1(dir_meta, is_train=False, is_eval=False):

    d_meta = {}
    file_list = []
    with open(dir_meta, "r") as f:
        l_meta = f.readlines()

    if is_train:
        for line in l_meta:
            _, key, _, _, label = line.strip().split(" ")
            file_list.append(key)
            d_meta[key] = 1 if label == "bonafide" else 0
        return d_meta, file_list

    elif is_eval:
        for line in l_meta:
            _, key, _, _, _ = line.strip().split(" ")
            #key = line.strip()
            file_list.append(key)
        return file_list
    else:
        for line in l_meta:
            _, key, _, _, label = line.strip().split(" ")
            file_list.append(key)
            d_meta[key] = 1 if label == "bonafide" else 0
        return d_meta, file_list

In [52]:
file_eval = genSpoof_list1(dir_meta=eval_trial_path,
                          is_train=False,
                          is_eval=True)
eval_set = Dataset_ASVspoof2019_devNeval(list_IDs=file_eval,
                                         base_dir=eval_database_path)
eval_loader = DataLoader(eval_set,
                         batch_size=config["batch_size"],
                         shuffle=False,
                         drop_last=False,
                         pin_memory=True)

In [56]:
eval_set[0]

(tensor([0.0009, 0.0008, 0.0007,  ..., 0.1776, 0.1562, 0.1233]),
 'LA_E_2834763')

In [57]:
eval_score_path = Path('exp_result/CMU_AASIST-L_ep100_bs24/eval_scores_cmu_asv')

In [75]:
eval_trial_path

PosixPath('/DATA/nfsshare/rishith/datasets/asvSpoof2019/DS_10283_3336/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt')

In [61]:
trial_path = eval_trial_path
with open(trial_path, "r") as f_trl:
    trial_lines = f_trl.readlines()
fname_list = []
score_list = []
for batch_x, utt_id in eval_loader:
    batch_x = batch_x.to(device)
    with torch.no_grad():
        _, batch_out = model(batch_x)
        batch_score = (batch_out[:, 1]).data.cpu().numpy().ravel()
    # add outputs
    fname_list.extend(utt_id)
    score_list.extend(batch_score.tolist())   
assert len(trial_lines) == len(fname_list) == len(score_list)

with open(eval_score_path, "w") as fh:
    for fn, sco, trl in zip(fname_list, score_list, trial_lines):
        _, utt_id, _, src, key = trl.strip().split(' ')
        assert fn == utt_id
        fh.write("{} {} {} {}\n".format(utt_id, src, key, sco))
print("Scores saved to {}".format(eval_score_path))

Scores saved to exp_result/CMU_AASIST-L_ep100_bs24/eval_scores_cmu_asv


In [64]:
cm_data = np.genfromtxt(eval_score_path, dtype=str)
cm_data[0]

array(['LA_E_2834763', 'A11', 'spoof', '-0.0017449557781219482'],
      dtype='<U22')

In [67]:
def getEER1(score_path):
    cm_data = np.genfromtxt(score_path, dtype=str)
    cm_keys = cm_data[:, 2]
    cm_scores = cm_data[:, 3].astype(float)
    bona_cm = cm_scores[cm_keys == 'bonafide']
    spoof_cm = cm_scores[cm_keys == 'spoof']
    eer_cm = compute_eer(bona_cm, spoof_cm)[0]
    return eer_cm * 100

In [68]:
eer_asv = getEER1(eval_score_path)
print('ASV EER: ', eer_asv)

ASV EER:  18.38219147297221


In [69]:
def produce_evaluation_file(
    data_loader: DataLoader,
    model,
    device: torch.device,
    save_path: str,
    trial_path: str) -> None:
    """Perform evaluation and save the score to a file"""
    model.eval()
    with open(trial_path, "r") as f_trl:
        trial_lines = f_trl.readlines()
    fname_list = []
    score_list = []
    for batch_x, utt_id in data_loader:
        batch_x = batch_x.to(device)
        with torch.no_grad():
            _, batch_out = model(batch_x)
            batch_score = (batch_out[:, 1]).data.cpu().numpy().ravel()
        # add outputs
        fname_list.extend(utt_id)
        score_list.extend(batch_score.tolist())

    assert len(trial_lines) == len(fname_list) == len(score_list)
    with open(save_path, "w") as fh:
        for fn, sco, trl in zip(fname_list, score_list, trial_lines):
            _, utt_id, _, src, key = trl.strip().split(' ')
            assert fn == utt_id
            fh.write("{} {} {} {}\n".format(utt_id, src, key, sco))
    print("Scores saved to {}".format(save_path))

In [72]:
produce_evaluation_file(eval_loader, model, device, eval_score_path,
                            SET_CM_PROTOCOL['eval'])

Scores saved to exp_result/CMU_AASIST-L_ep100_bs24/eval_scores_cmu_asv


In [73]:
eer_asv = getEER1(eval_score_path)
print('ASV EER: ', eer_asv)

ASV EER:  18.38219147297221
